# Consensus

Head slot tracking, justification, finalization, and fork choice analysis for PQ Devnet clients.

This notebook examines:
- Head slot vs current slot (how far behind each client is)
- Justified and finalized slot progression
- Finality lag (gap between head and finalized slot)
- Fork choice reorgs

In [ ]:
# Parameters - injected by papermill
devnet_id = None  # e.g., "pqdevnet-20260203T0100Z"

In [ ]:
import json
from pathlib import Path

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display

# Set default renderer for static HTML output
import plotly.io as pio
pio.renderers.default = "notebook"

In [ ]:
# Resolve devnet_id
DATA_DIR = Path("../data")

if devnet_id is None:
    # Use latest devnet from manifest
    devnets_path = DATA_DIR / "devnets.json"
    if devnets_path.exists():
        with open(devnets_path) as f:
            devnets = json.load(f).get("devnets", [])
        if devnets:
            devnet_id = devnets[-1]["id"]  # Latest
            print(f"Using latest devnet: {devnet_id}")
    else:
        raise ValueError("No devnets.json found. Run 'just detect-devnets' first.")

DEVNET_DIR = DATA_DIR / devnet_id
print(f"Loading data from: {DEVNET_DIR}")

In [ ]:
# Load devnet metadata
with open(DATA_DIR / "devnets.json") as f:
    devnets_data = json.load(f)
    devnet_info = next((d for d in devnets_data["devnets"] if d["id"] == devnet_id), None)

if devnet_info:
    print(f"Devnet: {devnet_info['id']}")
    print(f"Duration: {devnet_info['duration_hours']:.1f} hours")
    print(f"Time: {devnet_info['start_time']} to {devnet_info['end_time']}")
    print(f"Slots: {devnet_info['start_slot']} \u2192 {devnet_info['end_slot']}")
    print(f"Clients: {', '.join(devnet_info['clients'])}")

## Load Data

In [ ]:
# Load head slot data
head_df = pd.read_parquet(DEVNET_DIR / "head_slot.parquet")
# Deduplicate across instances
head_df = head_df.groupby(["client", "metric", "timestamp"], as_index=False)["value"].max()
print(f"Head slot: {len(head_df)} records, clients: {sorted(head_df['client'].unique())}")

# Load finality data
finality_df = pd.read_parquet(DEVNET_DIR / "finality_metrics.parquet")
finality_df = finality_df.groupby(["client", "metric", "timestamp"], as_index=False)["value"].max()
print(f"Finality: {len(finality_df)} records, clients: {sorted(finality_df['client'].unique())}")
print(f"Finality metrics: {sorted(finality_df['metric'].unique())}")

# Load fork choice reorgs
reorgs_df = pd.read_parquet(DEVNET_DIR / "fork_choice_reorgs.parquet")
reorgs_df = reorgs_df.groupby(["client", "timestamp"], as_index=False)["value"].max()
print(f"Reorgs: {len(reorgs_df)} records, clients: {sorted(reorgs_df['client'].unique())}")

## Head Slot vs Current Slot

Comparing each client's head slot (`lean_head_slot`) against the expected current slot (`lean_current_slot`). A gap indicates the client is falling behind.

In [ ]:
clients = sorted(head_df["client"].unique())
n_cols = min(len(clients), 2)
n_rows = -(-len(clients) // n_cols)

fig = make_subplots(
    rows=n_rows, cols=n_cols,
    subplot_titles=clients,
    vertical_spacing=0.12 / max(n_rows - 1, 1) * 2,
    horizontal_spacing=0.08,
)

colors = {"lean_head_slot": "#636EFA", "lean_current_slot": "#EF553B"}
labels = {"lean_head_slot": "head_slot", "lean_current_slot": "current_slot"}
legend_added = set()

for i, client in enumerate(clients):
    row = i // n_cols + 1
    col = i % n_cols + 1
    cdf = head_df[head_df["client"] == client]
    for metric in ["lean_current_slot", "lean_head_slot"]:
        mdf = cdf[cdf["metric"] == metric].sort_values("timestamp")
        if mdf.empty:
            continue
        label = labels[metric]
        show_legend = metric not in legend_added
        legend_added.add(metric)
        fig.add_trace(
            go.Scatter(
                x=mdf["timestamp"], y=mdf["value"],
                name=label, legendgroup=metric,
                showlegend=show_legend,
                line=dict(color=colors[metric]),
            ),
            row=row, col=col,
        )
    fig.update_yaxes(title_text="Slot", row=row, col=col)

fig.update_layout(
    title="Head Slot vs Current Slot by Client",
    height=270 * n_rows,
)
fig.show()

## Head Slot Lag

Difference between current slot and head slot. A value of 0 means the client is fully synced; higher values indicate falling behind.

In [ ]:
# Compute lag per client
current_df = head_df[head_df["metric"] == "lean_current_slot"][["client", "timestamp", "value"]].rename(columns={"value": "current_slot"})
head_only = head_df[head_df["metric"] == "lean_head_slot"][["client", "timestamp", "value"]].rename(columns={"value": "head_slot"})
lag_df = current_df.merge(head_only, on=["client", "timestamp"], how="inner")
lag_df["lag"] = lag_df["current_slot"] - lag_df["head_slot"]

fig = make_subplots(
    rows=n_rows, cols=n_cols,
    subplot_titles=clients,
    vertical_spacing=0.12 / max(n_rows - 1, 1) * 2,
    horizontal_spacing=0.08,
)

for i, client in enumerate(clients):
    row = i // n_cols + 1
    col = i % n_cols + 1
    cdf = lag_df[lag_df["client"] == client].sort_values("timestamp")
    fig.add_trace(
        go.Scatter(
            x=cdf["timestamp"], y=cdf["lag"],
            name=client, showlegend=False,
            line=dict(color="#636EFA"),
        ),
        row=row, col=col,
    )
    fig.update_yaxes(title_text="Slots behind", row=row, col=col)

fig.update_layout(
    title="Head Slot Lag (current_slot - head_slot)",
    height=270 * n_rows,
)
fig.show()

## Justification & Finalization

Progression of justified and finalized slots over time. Justified slots should track close to the head, and finalized slots should follow one epoch behind justified.

In [ ]:
# Use lean_latest_justified_slot and lean_latest_finalized_slot (continuous gauges)
jf_metrics = ["lean_latest_justified_slot", "lean_latest_finalized_slot"]
jf_df = finality_df[finality_df["metric"].isin(jf_metrics)].copy()

# Also include head_slot for reference
head_only_all = head_df[head_df["metric"] == "lean_head_slot"].copy()
head_only_all["metric"] = "lean_head_slot"

combined = pd.concat([jf_df, head_only_all], ignore_index=True)

fin_clients = sorted(combined["client"].unique())
n_cols_f = min(len(fin_clients), 2)
n_rows_f = -(-len(fin_clients) // n_cols_f)

fig = make_subplots(
    rows=n_rows_f, cols=n_cols_f,
    subplot_titles=fin_clients,
    vertical_spacing=0.12 / max(n_rows - 1, 1) * 2,
    horizontal_spacing=0.08,
)

colors = {
    "lean_head_slot": "#636EFA",
    "lean_latest_justified_slot": "#00CC96",
    "lean_latest_finalized_slot": "#EF553B",
}
labels = {
    "lean_head_slot": "head",
    "lean_latest_justified_slot": "justified",
    "lean_latest_finalized_slot": "finalized",
}
legend_added = set()

for i, client in enumerate(fin_clients):
    row = i // n_cols_f + 1
    col = i % n_cols_f + 1
    cdf = combined[combined["client"] == client]
    for metric in ["lean_head_slot", "lean_latest_justified_slot", "lean_latest_finalized_slot"]:
        mdf = cdf[cdf["metric"] == metric].sort_values("timestamp")
        if mdf.empty:
            continue
        show_legend = metric not in legend_added
        legend_added.add(metric)
        fig.add_trace(
            go.Scatter(
                x=mdf["timestamp"], y=mdf["value"],
                name=labels[metric], legendgroup=metric,
                showlegend=show_legend,
                line=dict(color=colors[metric]),
            ),
            row=row, col=col,
        )
    fig.update_yaxes(title_text="Slot", row=row, col=col)

fig.update_layout(
    title="Head, Justified & Finalized Slot by Client",
    height=270 * n_rows_f,
)
fig.show()

## Finality Lag

Gap between head slot and finalized slot. In healthy operation this should stay within 2 epochs (64 slots). Large or growing gaps indicate finality issues.

In [ ]:
# Compute finality lag: head_slot - finalized_slot
head_ts = head_df[head_df["metric"] == "lean_head_slot"][["client", "timestamp", "value"]].rename(columns={"value": "head_slot"})
fin_ts = finality_df[finality_df["metric"] == "lean_latest_finalized_slot"][["client", "timestamp", "value"]].rename(columns={"value": "finalized_slot"})
finality_lag = head_ts.merge(fin_ts, on=["client", "timestamp"], how="inner")
finality_lag["lag"] = finality_lag["head_slot"] - finality_lag["finalized_slot"]

fig = make_subplots(
    rows=n_rows_f, cols=n_cols_f,
    subplot_titles=fin_clients,
    vertical_spacing=0.12 / max(n_rows - 1, 1) * 2,
    horizontal_spacing=0.08,
)

for i, client in enumerate(fin_clients):
    row = i // n_cols_f + 1
    col = i % n_cols_f + 1
    cdf = finality_lag[finality_lag["client"] == client].sort_values("timestamp")
    fig.add_trace(
        go.Scatter(
            x=cdf["timestamp"], y=cdf["lag"],
            name=client, showlegend=False,
            line=dict(color="#636EFA"),
        ),
        row=row, col=col,
    )
    fig.update_yaxes(title_text="Slots behind", row=row, col=col)

fig.update_layout(
    title="Finality Lag (head_slot - finalized_slot)",
    height=270 * n_rows_f,
)
fig.show()

## Fork Choice Reorgs

Cumulative chain reorgs per client. Reorgs occur when the fork choice rule switches to a different chain head, often caused by late-arriving blocks or attestations.

In [ ]:
reorg_clients = sorted(reorgs_df["client"].unique())

if not reorg_clients:
    print("No reorg data available")
else:
    n_cols_r = min(len(reorg_clients), 2)
    n_rows_r = -(-len(reorg_clients) // n_cols_r)

    fig = make_subplots(
        rows=n_rows_r, cols=n_cols_r,
        subplot_titles=reorg_clients,
        vertical_spacing=0.12 / max(n_rows_r - 1, 1) * 2,
        horizontal_spacing=0.08,
    )

    for i, client in enumerate(reorg_clients):
        row = i // n_cols_r + 1
        col = i % n_cols_r + 1
        cdf = reorgs_df[reorgs_df["client"] == client].sort_values("timestamp")
        fig.add_trace(
            go.Scatter(
                x=cdf["timestamp"], y=cdf["value"],
                name=client, showlegend=False,
                line=dict(color="#636EFA"),
            ),
            row=row, col=col,
        )
        fig.update_yaxes(title_text="Cumulative reorgs", row=row, col=col)

    fig.update_layout(
        title="Fork Choice Reorgs by Client",
        height=270 * n_rows_r,
    )
    fig.show()

## Summary

In [ ]:
# Summary statistics per client
summary_rows = []

for client in sorted(head_df["client"].unique()):
    row = {"Client": client}

    # Head slot lag
    client_lag = lag_df[lag_df["client"] == client]["lag"]
    if not client_lag.empty:
        row["Avg Head Lag"] = f"{client_lag.mean():.1f}"
        row["Max Head Lag"] = f"{client_lag.max():.0f}"

    # Finality lag
    client_fin = finality_lag[finality_lag["client"] == client]["lag"]
    if not client_fin.empty:
        row["Avg Finality Lag"] = f"{client_fin.mean():.1f}"
        row["Max Finality Lag"] = f"{client_fin.max():.0f}"

    # Reorgs
    client_reorgs = reorgs_df[reorgs_df["client"] == client]["value"]
    if not client_reorgs.empty:
        row["Reorgs"] = f"{client_reorgs.max():.0f}"

    # Final head slot
    client_head = head_df[(head_df["client"] == client) & (head_df["metric"] == "lean_head_slot")]
    if not client_head.empty:
        row["Final Head Slot"] = f"{client_head['value'].max():.0f}"

    # Final finalized slot
    client_finalized = finality_df[(finality_df["client"] == client) & (finality_df["metric"] == "lean_latest_finalized_slot")]
    if not client_finalized.empty:
        row["Final Finalized Slot"] = f"{client_finalized['value'].max():.0f}"

    summary_rows.append(row)

if summary_rows:
    summary_df = pd.DataFrame(summary_rows).set_index("Client").fillna("-")
    display(summary_df)

print(f"\nDevnet: {devnet_id}")
if devnet_info:
    print(f"Duration: {devnet_info['duration_hours']:.1f} hours")